In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [3]:
def form_df(game_id,curr_df):
    url = "https://www.j-archive.com/showgame.php?game_id=" + str(game_id)
    response = requests.get(url)
    soup = bs(response.text, "html5lib")
    ps = soup.prettify()
    
    #Creating clue values and types
    #Clue values as read down the page
    clue_values = soup.find_all("td",{"class":re.compile("clue_value")})
    value_list = []
    for val in clue_values:
        money_val = re.search("\d+,?\d*",str(val)).group(0).replace(',','')
        #Get the type of clue
        #d for daily double, n for normal
        if 'daily' in str(val):
            #Daily double
            clue_type = 'd'
        else:
            clue_type  = 'n'
        value_list.append((money_val,clue_type))
    
    if value_list == []:
        return curr_df
                
    #Finding who answered right or wrong
    # 'r' for right, 'w' for wrong, 'n' for no response (not wrong just chose to not answer)
    contestants = []
    all_questions_responses = []
    for ele in ps.split('<'):
        if ('class=&' in ele):
            all_questions_responses.append(ele)
            for i in ele.split("class=&quot;"):
                if((("right&quot" in i) or ("wrong&quot" in i)) and ("div" not in i)):
                    #print(i)
                    try:
                        answerer_name = re.search("&gt;\w*\s?-?.?\w*!?.?&lt",str(i)).group(0).replace("&gt;",'').replace("&lt",'')
                    except:
                        print("Nonetype object returned.",i)
                    if answerer_name not in contestants and answerer_name.lower() != 'triple stumper':
                        contestants.append(answerer_name)
                        if len(contestants) > 3:
                            print("Too many contestants")
                            return curr_df

    contestant_responses = [] #Gonna be list of lists of length 3
    for responses in all_questions_responses:
        qresponses = ['','','']
        for i in responses.split("class=&quot;"):
            if((("right&quot" in i) or ("wrong&quot" in i)) and ("div" not in i)):
                try:
                    answerer_name = re.search("&gt;\w*\s?-?.?\w*!?.?&lt",str(i)).group(0).replace("&gt;",'').replace("&lt",'')
                except:
                    print("Nonetype object returned.",i)
                #print(answerer_name)
                if 'triple' in answerer_name.lower() or 'stumper' in answerer_name.lower():
                    #Nobody got it
                    #Any non-answerers should be set to 'n'
                    for repo_num,repo in enumerate(qresponses):
                        if repo == '':
                            #No response yet, set to 'n'
                            qresponses[repo_num] = 'n'
                        #Otherwise, just leave it be
                elif("right&quot" in i):
                    #This person got it right
                    for contestant_num,contestant in enumerate(contestants):
                        if answerer_name == contestant:
                            #This person got it right, set their qresponse to 'r'
                            qresponses[contestant_num] = 'r'
                else:
                    #Must be wrong
                    for contestant_num,contestant in enumerate(contestants):
                        if answerer_name == contestant:
                            #This person got it wrong, set their qresponse to 'w'
                            qresponses[contestant_num] = 'w'
        #Final cleanup
        for num,i in enumerate(qresponses):
            if i == '':
                qresponses[num] = 'n'
        contestant_responses.append(qresponses)
        
    ##Final Jeopardy + Tiebreaker
    if 'clue_TB' in str(all_questions_responses[-1]):
        ##This is the tiebreaker
        
        final_jeopardy_responses = contestant_responses[-2]
        final_jeopardy_soup = all_questions_responses[-2]
        
        #Use the tiebreaker to find the singular winner
        for i in str(all_questions_responses[-1]).split("class=&quot;"):
            if((("right&quot" in i) or ("wrong&quot" in i)) and ("div" not in i)):
                answerer_name = re.search("&gt;\w*\s?-?.?\w*!?.?&lt",str(i)).group(0).replace("&gt;",'').replace("&lt",'')
        #Have their name, now look for it and get that index
        win_index = contestants.index(answerer_name)
        winner_list = []
        for i in range(3):
            if i == win_index:
                winner_list.append(True)
            else:
                winner_list.append(False)
    else:
        #No tiebreaker
        final_jeopardy_responses = contestant_responses[-1]
        final_jeopardy_soup = all_questions_responses[-1]
        #Get the winner
        final_results = soup.find_all("td",{"class":re.compile("score_positive")})
        final_results = final_results[-6:-3]
        final_money_values = []
        for val in final_results:
            money_val = re.search("\d+,?\d*",str(val)).group(0).replace(',','')
            final_money_values.append(int(money_val))

        score_max = max(final_money_values)
        max_count = 0
        for i in final_money_values:
            if score_max == i:
                max_count += 1
        #For a winner, should be 1
        #Tie on 2
        #No winner on 3
        winner_list = []
        #Adjust winner traits
        if max_count == 3:
            winner_list = [False,False,False]
        else:
            #print(game_id,"TIED!")
            #Co-Winners apparently
            winning_score = max(final_money_values)
            for ele in final_money_values:
                if winning_score == ele:
                    winner_list.append(True)
                else:
                    winner_list.append(False)

    #Final Jeopardy Values
    final_jeopardy_values = [0,0,0]
    #print(final_jeopardy_responses)
    for i in final_jeopardy_soup.split("class=&quot;"):
            if((("right&quot" in i) or ("wrong&quot" in i)) and ("div" not in i)):
                try:
                    answerer_name = re.search("&gt;\w*\s?-?.?\w*!?.?&lt",str(i)).group(0).replace("&gt;",'').replace("&lt",'')
                except:
                    print("Nonetype object returned.",i)

                try:
                    betted_amount = re.search("gt;\$?\d+,?\d*",str(i)).group(0).replace("$","").replace(",","").replace("gt;","")
                except:
                    return curr_df
                for contestant_num,contestant in enumerate(contestants):
                    if answerer_name == contestant:
                        final_jeopardy_values[contestant_num] = betted_amount

    
    #Incorrect and Correct responses counting
    #Answering frequency
    unique_values = [100,200,300,400,500,600,800,1000,1200,1600,2000]
    #contestant counts of questions answered in each unique value
    unique_counts_correct = [[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0]]
    unique_counts_incorrect = [[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0]]
    pre_final_jeopardy_scores = [0,0,0]
    
    #Fill in the answering frequency and scores
    for index,clue in enumerate(value_list):
        #print(clue,index)
        if clue[1] == 'n':
            #Normal question
            clue_value = clue[0]
            try:
                unique_index = unique_values.index(int(clue_value))
            except:
                #We got some other value than in the unique_values
                #Basically this was a game with a different amount of money per clue than the standard, so we'll ignore it
                return curr_df
            #Otherwise, has a normal value
            for response_index,response in enumerate(contestant_responses[index]):
                if response == 'r':
                    #This player got it right
                    unique_counts_correct[response_index][unique_index] += 1
                    pre_final_jeopardy_scores[response_index] += int(unique_values[unique_index])
                elif response == 'w':
                    unique_counts_incorrect[response_index][unique_index] += 1
                    pre_final_jeopardy_scores[response_index] -= int(unique_values[unique_index])

                    
                    
    #CREATE THE DATAFRAME
    #Begin the current game dataframe
    jeopardy_df = pd.DataFrame()
    jeopardy_df['Winner'] = winner_list
    jeopardy_df['Final Jeopardy Bets'] = final_jeopardy_values
    jeopardy_df['Final Jeopardy Result'] = final_jeopardy_responses
    print(final_jeopardy_responses)
    
    
    DD_count = 1 #Tracks the current daily double
    
    #Fill dataframe with daily double values
    for index,clue in enumerate(value_list):
        if clue[1] == 'd':
            #print(clue)
            #Was a daily double
            #Form the daily double data
            DD_bet_list = [None,None,None]
            DD_bet_col_name = 'Daily Double ' + str(DD_count) + ' Bet'
            DD_results_col_name = 'Daily Double ' + str(DD_count) + ' Result'
            for response_index,DD_result in enumerate(contestant_responses[index]):
                if DD_result == 'r':
                    pre_final_jeopardy_scores[response_index] += int(value_list[index][0])
                    DD_bet_list[response_index] = value_list[index][0]
                elif DD_result == 'w':
                    pre_final_jeopardy_scores[response_index] -= int(value_list[index][0])
                    DD_bet_list[response_index] = value_list[index][0]
            jeopardy_df[DD_bet_col_name] = DD_bet_list
            jeopardy_df[DD_results_col_name] = contestant_responses[index]
            DD_count +=1

    ##Now add in the counts of answering wrong/right
    #Start with getting all the column names
    incorrect_counts_columns = []
    correct_counts_columns = []
    for value in unique_values:
        val = str(value)
        colname = "Count Correct " + val + " Valued Questions"
        correct_counts_columns.append(colname)
        colname = "Count Incorrect " + val + " Valued Questions"
        incorrect_counts_columns.append(colname)
    
    jeopardy_df["Pre-Final Jeopardy Scores"] = pre_final_jeopardy_scores
    
    correct_counts_df = pd.DataFrame(data = unique_counts_correct,columns=correct_counts_columns)
    incorrect_counts_df = pd.DataFrame(data = unique_counts_incorrect,columns=incorrect_counts_columns)

    counts_df = pd.concat([correct_counts_df,incorrect_counts_df],axis=1)
    final_df = pd.concat([jeopardy_df,counts_df],axis=1)
    
    return pd.concat([final_df,curr_df])

In [4]:
max_game_id = 7201
min_game_id = 173

In [ ]:
df = pd.DataFrame() #Empty data frame
for i in range(min_game_id,max_game_id+1):
    if i%1000 == 0:
        print(i)
    #3575-3576 are a strange IBM game, ignore
    if i == 3575 or i == 3576:
        pass
    else:
        try:
            df = form_df(i,df) #Will call with an empty frame, create a frame, then add more to that until done
        except:
            print("Error at id:",i)
df

['r', 'r', 'r']
['w', 'w', 'w']
['w', 'r', 'w']
['n', 'r', 'r']
['w', 'r', 'n']
['w', 'r', 'w']
['w', 'w', 'w']
['r', 'w', 'r']
['w', 'r', 'r']
['w', 'r', 'r']
['w', 'r', 'r']
['r', 'w', 'w']
Error at id: 185
['r', 'w', 'r']
['r', 'r', 'w']
['w', 'r', 'r']
['w', 'w', 'w']
['r', 'r', 'r']
['r', 'r', 'w']
['r', 'w', 'r']
Error at id: 193
Error at id: 194
Error at id: 195
['r', 'w', 'w']
['w', 'r', 'r']
Error at id: 198
Error at id: 199
Error at id: 200
Error at id: 201
Error at id: 202
['r', 'r', 'r']
['w', 'r', 'r']
['r', 'w', 'w']
['r', 'r', 'r']
['r', 'w', 'r']
['r', 'w', 'w']
['w', 'r', 'r']
['r', 'r', 'r']
['r', 'w', 'w']
['r', 'r', 'r']
['w', 'w', 'r']


In [37]:
df.to_csv('Jeopardy_df.csv',index=False)